In [1]:
import findspark
findspark.init()

import pyspark

import re

from pyspark import SparkConf, SparkContext
sc = pyspark.SparkContext(appName = "MovieLens")

from pyspark.sql import HiveContext,SQLContext

sqlContext = SQLContext.getOrCreate(sc)

# from pyspark import HiveConf, HiveContext
# sqlContext = pyspark.HiveContext(appName = "MovieLens")

# from pyspark.sql import *


sc.addPyFile("pairs.py")
import pairs

from pairs import FindPairs
from pairs import iterate

lines = sc.textFile("ratings")
data = lines.map(lambda gen: re.split(r',', gen))
data1 = data.map(lambda x: (x[0],(x[1],x[2])))
#data2 = data1.sortByKey()

pairwise= data1.filter(lambda p: len(p[1]) > 1)
pairwise= pairwise.map(lambda p: FindPairs(p[0],p[1]))

#inmem = pairwise2.persist()

#inmem.saveAsTextFile("PAIRS4")


In [ ]:
pairwise1= pairwise.mapPartitions(lambda p: FindPairs(p[0],p[1]))

pairwise2=pairwise1.flatMap(lambda p: p).groupByKey().mapValues(iterate)

rdd_data = sqlContext.createDataFrame(pairwise2, ["moviepairs","ratings"])

rdd_data.show(5)

In [3]:
''' 
Obtain the sparse object-item matrix:
    user_id -> [(object_id_1, rating_1),
               [(object_id_2, rating_2),
                ...]
'''
object_item_pairs = lines.map(parseVector).groupByKey().map(
    lambda p: sampleInteractions(p[0],p[1],500)).cache()


'''
Get all item-item pair combos:
    (item1,item2) ->    [(item1_rating,item2_rating),
                         (item1_rating,item2_rating),
                         ...]
'''

pairwise_objects = object_item_pairs.filter(
    lambda p: len(p[1]) > 1).map(
    lambda p: findItemPairs(p[0],p[1])).groupByKey()



x = pairwise_objects.mapValues(iterate)
x.collect()

Py4JJavaError: An error occurred while calling o90.saveAsTextFile.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 4 times, most recent failure: Lost task 0.3 in stage 2.0 (TID 11, staff77.elec.qmul.ac.uk, executor 3): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/cloudera/parcels/CDH/lib/spark/python/pyspark/worker.py", line 111, in main
    process()
  File "/opt/cloudera/parcels/CDH/lib/spark/python/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/lib/spark/python/pyspark/rdd.py", line 2346, in pipeline_func
  File "/usr/lib/spark/python/pyspark/rdd.py", line 2346, in pipeline_func
  File "/usr/lib/spark/python/pyspark/rdd.py", line 2346, in pipeline_func
  File "/usr/lib/spark/python/pyspark/rdd.py", line 2346, in pipeline_func
  File "/usr/lib/spark/python/pyspark/rdd.py", line 317, in func
  File "<ipython-input-1-7ba